In [1]:
# here will be cleaning the data 
import os 
import pathlib
import json
import pandas as pd
import numpy as np

In [2]:
class Review:
    CsvData = {'comment': [],'rating':[] }
    def __init__(self, comment:str, rating: float):
        self.comment = comment
        self.rating = self.get_rate(rating)
    
    @classmethod
    def pushtocsv(cls,comment:str, rating:float):
        cls.CsvData['comment'].append(comment)  
        cls.CsvData['rating'].append(rating)
        
    
    def get_rate(self, value:float):
        if value <= 2:
            return "NEGATIVE"
        elif value == 3:
            return "NEUTRAL"
        elif value >= 4:
            return "POSITIVE"
        
    @classmethod
    def saveTheToCsv(cls, path):
        s = os.path.join(path, 'ratings.csv')
        df = pd.DataFrame(cls.CsvData, columns= {'comment','rating'})
#         df.to_csv(s)


        

In [3]:
data = []
def formatData(Datapath: str, PreparedDataPath="cleanedData"):
    Datapath = pathlib.Path(Datapath)
    savingDir = pathlib.Path(PreparedDataPath)
    for file in Datapath.iterdir():
        with open(file) as f:
            for line in f:
                review = json.loads(line)
                reviewObject = Review(review['reviewText'], review['overall'])
                data.append(reviewObject)
                Review.pushtocsv(review['reviewText'], review['overall'])
    
#     Review.saveTheToCsv(savingDir)
                
                


In [4]:
formatData("data")

In [5]:
# creating the dataset 
from sklearn.model_selection import train_test_split
training, testing = train_test_split(data, test_size=0.33, train_size=0.66, random_state=4)
# now we split again the model int some of the categories

In [6]:
# destructuring the dataset for the given problem
training_x = [x.comment for x in training]
training_y = [x.rating for x in training]
testing_x = [x.comment for x in testing]
testing_y = [x.rating for x in testing]

In [7]:
# transformin the text int binaries using Bag of word Technique
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
trainingx = vectorizer.fit_transform(training_x)
testingx = vectorizer.transform(testing_x)
print(f"Num of Words: {len(vectorizer.get_feature_names_out())}")
trainingx.toarray()[0]

Num of Words: 29187


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [8]:
# Training the Model and testing it
# Classifier svm and BAYES Decison Tree Naive Bayes and LGR


from sklearn import svm

cl_svm = svm.SVC(kernel='linear')

cl_svm.fit(trainingx,training_y)


SVC(kernel='linear')

In [80]:
# Using the Naive Bayes Classifier
from sklearn.naive_bayes import GaussianNB
ngb = GaussianNB()
ngb.fit(trainingx.todense(), testing_y )

d:\python37\lib\site-packages\sklearn\utils\validation.py:598: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  FutureWarning,


ValueError: Found input variables with inconsistent numbers of samples: [1980, 990]

In [9]:
# using decison tree 

from sklearn.tree import DecisionTreeClassifier
cls_t = DecisionTreeClassifier()
cls_t.fit(trainingx, training_y)

DecisionTreeClassifier()

In [10]:
# Prdiction 

# cls_t.predict(testingx[126])
from sklearn.ensemble import RandomForestClassifier
cls = RandomForestClassifier()
cls.fit(trainingx, training_y)

RandomForestClassifier()

In [11]:
# print(testing_x[126],testing_y[126] )
# testing the accuracy

print(cls_t.score(trainingx, training_y))
print(cl_svm.score(trainingx, training_y))
print(cls.score(trainingx, training_y))

1.0
0.9968531468531469
1.0


In [14]:
# Checking the Accuracy on each of the model and  data

from sklearn.metrics import f1_score

f1_score(testing_y,cl_svm.predict(testingx), average= None , labels=['POSITIVE', 'NEUTRAL', 'NEGATIVE'])

array([0.90165312, 0.29893238, 0.39732888])